# Проведите дисперсионный анализ сегментации клиентов
## Условие задачи

Вы работаете в онлайн-школе аналитиком. Вы знаете, что все клиенты школы были разделены на сегменты A/B/C/D на основе экспертной оценки. Это сделано, чтобы обеспечить высокий уровень клиентского сервиса.

Сейчас вы хотите оказывать клиентскую поддержку различным группам пользователей в зависимости от их возраста и наличия высшего образования. Ваша задача оценить, подходит ли текущая сегментация пользователей для новых задач или ее необходимо переделать.

**Формальное описание задачи:**

Оценить, влияет ли возраст и уровень образования на сегмент пользователя.

**Алгоритм решения:**

1. Загрузите данные из файла формата CSV.
2. Проведите предварительный анализ данных: изучите структуру набора данных, определите переменные, которые необходимо будет исследовать. Удалите наблюдения, в которых необходимые переменные пропущены.
3. Проведите One-way ANOVA и Two-way ANOVA.
4. Проверьте ограничения на применимость методов: нормальность и гомоскедастичность. Независимость наблюдений гарантируется.
5. Проведите апостериорный анализ при наличии статистически значимых результатов.
6. Сделайте выводы о влиянии уровня образования и возраста на сегмент пользователя.

**Входные данные:**

[Customer Segmentation.csv](https://github.com/Eduson-DataScience/EDA-Practice/blob/main/Define_data_taxonomy_in_different_variations%20copy%205/Customer%20Segmentation.csv)

## Подсказки

- Проверьте, что у вас есть файл с данными (Customer Segmentation.csv) и он находится в том же каталоге, что и ваш скрипт.
- Используйте метод `ols` из библиотеки statsmodels для проведения дисперсионного анализа.
- Для проверки нормальности остатков воспользуйтесь методом построения графика `qqplot` из библиотеки statsmodels, а для проверки гомоскедастичности остатков – `bartlett` из библиотеки scipy.
- Не забудьте провести апостериорный анализ для выявления групп, между которыми есть отличия в средних значениях.

In [ ]:
import pandas as pd
import seaborn as sns

from scipy import stats

# anova model from formula
from statsmodels.formula.api import ols
# anova table from ols model
from statsmodels.stats.anova import anova_lm
# for qqplot
import statsmodels.api as sm
# tukey test
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# загружаем данные
data = pd.read_csv('Customer Segmentation.csv', index_col=0)
data.head()

data.info()

# удаляем наблюдения с пропущенными значениями
data = data[data.Graduated.isna() != True]

sns.boxplot(data=data, x='Segmentation', y='Age', hue='Graduated');

#one-way anova
one_way = ols('Age ~ Segmentation', data).fit()
anova_lm(one_way)

#qq-plot, проверяем нормальность остатков
sm.qqplot(one_way.resid, line='s');

#разделяем данные по сегментам, чтобы провести тест Левена
groups = []
for group_id, group in data.groupby(['Segmentation']):
    groups.append(group.Age)

#тест Левена
stats.bartlett(*groups)

#апостериорный анализ
print(pairwise_tukeyhsd(endog=data.Age, groups=data.Segmentation, alpha=0.05))

#two-way anova
two_way = ols('Age ~ Segmentation + Graduated + Segmentation:Graduated', data).fit()
anova_lm(two_way)

#qq-plot, проверяем нормальность остатков
sm.qqplot(two_way.resid, line='s');

#разделяем данные по сегментам и образованию, чтобы провести тест Левена
groups = []
for group_id, group in data.groupby(['Segmentation', 'Graduated']):
    groups.append(group.Age)

#тест Левена
stats.bartlett(*groups)

#апостериорный анализ
data['combination'] = data.Segmentation + '/' + data.Graduated
print(pairwise_tukeyhsd(endog=data.Age.astype('float'), groups=data.combination, alpha=0.05))

Из проведенного анализа данных можно сделать следующие выводы:

1. Возраст пользователя имеет статистическую значимость при разделении на сегменты. Также возраст пользователя имеет статистическую значимость при разделении сегментов на уровни образования в них.
2. Для нового типа клиентской поддержки нет необходимости менять текущую сегментацию пользователей, достаточно будет только подразделить уже существующие сегменты по уровню образования.